In [ ]:
! pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=b574af66af180a54163a1db6679ecc72dbf84be7c3bebe6be0aadefd03b5aa06
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [ ]:
import pyspark
import os, sys
from pyspark.sql import SparkSession
from pyspark import SparkContext

In [ ]:
spark  = SparkSession.builder.config("spark.driver.memory", '4g').appName('Kmeans').getOrCreate()


In [ ]:
! unzip kddcup.zip

Archive:  kddcup.zip
  inflating: kddcup.data_10_percent_corrected  


In [ ]:
data_without_header = spark.read.option('inferSchema', True).option('header', False).csv('kddcup.data_10_percent_corrected')
data_without_header

DataFrame[_c0: int, _c1: string, _c2: string, _c3: string, _c4: int, _c5: int, _c6: int, _c7: int, _c8: int, _c9: int, _c10: int, _c11: int, _c12: int, _c13: int, _c14: int, _c15: int, _c16: int, _c17: int, _c18: int, _c19: int, _c20: int, _c21: int, _c22: int, _c23: int, _c24: double, _c25: double, _c26: double, _c27: double, _c28: double, _c29: double, _c30: double, _c31: int, _c32: int, _c33: double, _c34: double, _c35: double, _c36: double, _c37: double, _c38: double, _c39: double, _c40: double, _c41: string]

In [ ]:
column_names = [  "duration", "protocol_type", "service", "flag",
  "src_bytes", "dst_bytes", "land", "wrong_fragment", "urgent",
  "hot", "num_failed_logins", "logged_in", "num_compromised",
  "root_shell", "su_attempted", "num_root", "num_file_creations",
  "num_shells", "num_access_files", "num_outbound_cmds",
  "is_host_login", "is_guest_login", "count", "srv_count",
  "serror_rate", "srv_serror_rate", "rerror_rate", "srv_rerror_rate",
  "same_srv_rate", "diff_srv_rate", "srv_diff_host_rate",
  "dst_host_count", "dst_host_srv_count",
  "dst_host_same_srv_rate", "dst_host_diff_srv_rate",
  "dst_host_same_src_port_rate", "dst_host_srv_diff_host_rate",
  "dst_host_serror_rate", "dst_host_srv_serror_rate",
  "dst_host_rerror_rate", "dst_host_srv_rerror_rate",
"label"]

data = data_without_header.toDF(*column_names)
data

DataFrame[duration: int, protocol_type: string, service: string, flag: string, src_bytes: int, dst_bytes: int, land: int, wrong_fragment: int, urgent: int, hot: int, num_failed_logins: int, logged_in: int, num_compromised: int, root_shell: int, su_attempted: int, num_root: int, num_file_creations: int, num_shells: int, num_access_files: int, num_outbound_cmds: int, is_host_login: int, is_guest_login: int, count: int, srv_count: int, serror_rate: double, srv_serror_rate: double, rerror_rate: double, srv_rerror_rate: double, same_srv_rate: double, diff_srv_rate: double, srv_diff_host_rate: double, dst_host_count: int, dst_host_srv_count: int, dst_host_same_srv_rate: double, dst_host_diff_srv_rate: double, dst_host_same_src_port_rate: double, dst_host_srv_diff_host_rate: double, dst_host_serror_rate: double, dst_host_srv_serror_rate: double, dst_host_rerror_rate: double, dst_host_srv_rerror_rate: double, label: string]

In [ ]:
from pyspark.sql.functions import col
data.select('label').groupBy('label').count().orderBy(col('count').desc()).show(25)

+----------------+------+
|           label| count|
+----------------+------+
|          smurf.|280790|
|        neptune.|107201|
|         normal.| 97278|
|           back.|  2203|
|          satan.|  1589|
|        ipsweep.|  1247|
|      portsweep.|  1040|
|    warezclient.|  1020|
|       teardrop.|   979|
|            pod.|   264|
|           nmap.|   231|
|   guess_passwd.|    53|
|buffer_overflow.|    30|
|           land.|    21|
|    warezmaster.|    20|
|           imap.|    12|
|        rootkit.|    10|
|     loadmodule.|     9|
|      ftp_write.|     8|
|       multihop.|     7|
|            phf.|     4|
|           perl.|     3|
|            spy.|     2|
+----------------+------+



In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.clustering import KMeans, KMeansModel
from pyspark.ml import Pipeline
numeric_only_df = data.drop("protocol_type", "service", "flag").cache()
assembler_df = VectorAssembler().setInputCols(numeric_only_df.columns[:-1]).setOutputCol('featureVector')

In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.clustering import KMeans, KMeansModel
from pyspark.ml import Pipeline
numeric_only = data.drop("protocol_type", "service", "flag").cache()
assembler = VectorAssembler().setInputCols(numeric_only.columns[:-1]).\
                              setOutputCol("featureVector")
kmeans = KMeans().setPredictionCol("cluster").setFeaturesCol('featureVector')
pipeline = Pipeline().setStages([assembler,kmeans ])
pipeline_model = pipeline.fit(numeric_only)
kmeans_model = pipeline_model.stages[1]
from pprint import pprint
pprint(kmeans_model.clusterCenters())


In [ ]:
kmeans_model = pipeline_model.stages[1]
from pprint import pprint
pprint(kmeans_model.clusterCenters())


[array([4.79793956e+01, 1.62207883e+03, 8.68534183e+02, 4.45326100e-05,
       6.43293794e-03, 1.41694668e-05, 3.45168212e-02, 1.51815716e-04,
       1.48247035e-01, 1.02121372e-02, 1.11331525e-04, 3.64357718e-05,
       1.13517671e-02, 1.08295211e-03, 1.09307315e-04, 1.00805635e-03,
       0.00000000e+00, 0.00000000e+00, 1.38658354e-03, 3.32286248e+02,
       2.92907143e+02, 1.76685418e-01, 1.76607809e-01, 5.74330999e-02,
       5.77183920e-02, 7.91548844e-01, 2.09816404e-02, 2.89968625e-02,
       2.32470732e+02, 1.88666046e+02, 7.53781203e-01, 3.09056111e-02,
       6.01935529e-01, 6.68351484e-03, 1.76753957e-01, 1.76441622e-01,
       5.81176268e-02, 5.74111170e-02]),
 array([2.0000000e+00, 6.9337564e+08, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 1.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00

In [ ]:
pprint(kmeans_model.summary.trainingCost)


2220633804525732.0


# Feature Normailization


In [ ]:
from pyspark.ml.feature import StandardScaler

def clustering_score_2(input_data,k):
  input_numeric_only = input_data.drop("protocol_type", "service", "flag")
  assembler = VectorAssembler().setInputCols(input_numeric_only.columns[:-1]).setOutputCol("featureVector")
  scaler = StandardScaler().setInputCol("featureVector").setoutputCol("scaledFeatureVector").setWithStd(True).setWithMean(False)
  kmeans = KMeans().setSeed(randint(100, 100000)).setK(k).setMaxIter(40).setTol(1.0e-5).setPredictionCol('Clusters').setFeatureCol("scaledFeatureVector")
  pipeline = Pipeline.setStages([assembler, scaler, kmeans])
  pipleine_model = pipeline.fit(input_numeric_only)
  kmeans_model = pipeline.stages[-1]
  training_cost = kmeans_model.summary.trainingCost
  return training_cost

for k in list(range(60, 271, 30)):
  print(k, clustering_score_2(numeric_only, k))


60 2220633804525732.0
90 2220633804525732.0
120 2220633804525732.0
150 2220633804525732.0
180 2220633804525732.0
210 2220633804525732.0
240 2220633804525732.0
270 2220633804525732.0


In [ ]:
! curl -L -o donation.zip https://bit.ly/1Aoywaq
! unzip donation.zip
! unzip 'block_*.zip'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   163  100   163    0     0   1222      0 --:--:-- --:--:-- --:--:--  1225
100 53.8M    0 53.8M    0     0   973k      0 --:--:--  0:00:56 --:--:-- 2326k
Archive:  donation.zip
 extracting: block_10.zip            
 extracting: block_1.zip             
 extracting: block_2.zip             
 extracting: block_3.zip             
 extracting: block_4.zip             
 extracting: block_5.zip             
 extracting: block_6.zip             
 extracting: block_7.zip             
 extracting: block_8.zip             
 extracting: block_9.zip             
  inflating: documentation           
  inflating: frequencies.csv         
Archive:  block_5.zip
  inflating: block_5.csv             

Archive:  block_4.zip
  inflating: block_4.csv             

Archive:  block_7.zip
  inflating: block_7.csv             

Archive:  block_2.zip


In [ ]:
parsed  = spark.read.option("inferSchema", True).option("header", True).option("nullValue", "?").csv("block_1.csv")

In [ ]:
parsed.show(5)

+-----+-----+-----------------+------------+------------+------------+-------+------+------+------+-------+--------+
| id_1| id_2|     cmp_fname_c1|cmp_fname_c2|cmp_lname_c1|cmp_lname_c2|cmp_sex|cmp_bd|cmp_bm|cmp_by|cmp_plz|is_match|
+-----+-----+-----------------+------------+------------+------------+-------+------+------+------+-------+--------+
|37291|53113|0.833333333333333|        NULL|         1.0|        NULL|      1|     1|     1|     1|      0|    true|
|39086|47614|              1.0|        NULL|         1.0|        NULL|      1|     1|     1|     1|      1|    true|
|70031|70237|              1.0|        NULL|         1.0|        NULL|      1|     1|     1|     1|      1|    true|
|84795|97439|              1.0|        NULL|         1.0|        NULL|      1|     1|     1|     1|      1|    true|
|36950|42116|              1.0|        NULL|         1.0|         1.0|      1|     1|     1|     1|      1|    true|
+-----+-----+-----------------+------------+------------+-------

In [ ]:
parsed.printSchema()

root
 |-- id_1: integer (nullable = true)
 |-- id_2: integer (nullable = true)
 |-- cmp_fname_c1: double (nullable = true)
 |-- cmp_fname_c2: double (nullable = true)
 |-- cmp_lname_c1: double (nullable = true)
 |-- cmp_lname_c2: double (nullable = true)
 |-- cmp_sex: integer (nullable = true)
 |-- cmp_bd: integer (nullable = true)
 |-- cmp_bm: integer (nullable = true)
 |-- cmp_by: integer (nullable = true)
 |-- cmp_plz: integer (nullable = true)
 |-- is_match: boolean (nullable = true)



In [ ]:
parsed.groupBy("is_match").count().orderBy(col("count").desc()).show()

NameError: name 'col' is not defined

In [ ]:
summary = parsed.describe()
summary.select("summary","cmp_fname_c1", "cmp_fname_c2").show()

+-------+------------------+------------------+
|summary|      cmp_fname_c1|      cmp_fname_c2|
+-------+------------------+------------------+
|  count|            574811|             10325|
|   mean|0.7127592938252765|0.8977586763518972|
| stddev|0.3889286452463553|0.2742577520430534|
|    min|               0.0|               0.0|
|    max|               1.0|               1.0|
+-------+------------------+------------------+



In [ ]:
parsed.createOrReplaceTempView("linkage")

In [ ]:
spark.sql("""
SELECT is_match , COUNT(*) as cnt
FROM linkage
GROUP BY is_match
ORDER BY cnt
""").show()

In [ ]:
matchs = parsed.where('is_match = true')
matchs_summary = matchs.describe()
matchs_summary.show()

In [ ]:
missed = parsed.filter(col('is_match')==False)
missed_summary = missed.describe()
missed_summary.show()

+-------+------------------+------------------+------------------+------------------+-------------------+-------------------+------------------+-------------------+------------------+-------------------+--------------------+
|summary|              id_1|              id_2|      cmp_fname_c1|      cmp_fname_c2|       cmp_lname_c1|       cmp_lname_c2|           cmp_sex|             cmp_bd|            cmp_bm|             cmp_by|             cmp_plz|
+-------+------------------+------------------+------------------+------------------+-------------------+-------------------+------------------+-------------------+------------------+-------------------+--------------------+
|  count|            572820|            572820|            572720|             10197|             572820|                186|            572820|             572758|            572758|             572758|              571529|
|   mean|33267.691156035056| 66621.93907335638|0.7117214109571942| 0.896531309395388|0.3130805753254

In [ ]:
summary_p = summary.toPandas()
summary_p

,summary,id_1,id_2,cmp_fname_c1,cmp_fname_c2,cmp_lname_c1,cmp_lname_c2,cmp_sex,cmp_bd,cmp_bm,cmp_by,cmp_plz
0,count,574913,574913,574811,10325,574913,239,574913,574851,574851,574851,573618
1,mean,33271.962171667714,66564.6636865056,0.7127592938252765,0.8977586763518972,0.31557245780987964,0.32691554145529067,0.9550923357099248,0.22475563232907309,0.4886361857246487,0.22266639529199742,0.0054949461139643455
2,stddev,23622.66942593358,23642.00230967225,0.3889286452463553,0.2742577520430534,0.3342494687554251,0.37830920205406704,0.20710152240504406,0.4174216587235586,0.49987128182816276,0.416036504164562,0.07392402321301904
3,min,1,6,0.0,0.0,0.0,0.0,0,0,0,0,0
4,max,99894,100000,1.0,1.0,1.0,1.0,1,1,1,1,1


In [ ]:
summary_p = summary_p.set_index('summary').transpose().reset_index()
summary_p


summary,index,count,mean,stddev,min,max
0,id_1,574913,33271.962171667714,23622.66942593358,1,99894
1,id_2,574913,66564.6636865056,23642.00230967225,6,100000
2,cmp_fname_c1,574811,0.7127592938252765,0.3889286452463553,0.0,1.0
3,cmp_fname_c2,10325,0.8977586763518972,0.2742577520430534,0.0,1.0
4,cmp_lname_c1,574913,0.31557245780987964,0.3342494687554251,0.0,1.0
5,cmp_lname_c2,239,0.32691554145529067,0.37830920205406704,0.0,1.0
6,cmp_sex,574913,0.9550923357099248,0.20710152240504406,0,1
7,cmp_bd,574851,0.22475563232907309,0.4174216587235586,0,1
8,cmp_bm,574851,0.4886361857246487,0.49987128182816276,0,1
9,cmp_by,574851,0.22266639529199742,0.416036504164562,0,1


In [ ]:
summary_p = summary_p.rename(columns={'index':'fields'})
summary_p

summary,fields,count,mean,stddev,min,max
0,id_1,574913,33271.962171667714,23622.66942593358,1,99894
1,id_2,574913,66564.6636865056,23642.00230967225,6,100000
2,cmp_fname_c1,574811,0.7127592938252765,0.3889286452463553,0.0,1.0
3,cmp_fname_c2,10325,0.8977586763518972,0.2742577520430534,0.0,1.0
4,cmp_lname_c1,574913,0.31557245780987964,0.3342494687554251,0.0,1.0
5,cmp_lname_c2,239,0.32691554145529067,0.37830920205406704,0.0,1.0
6,cmp_sex,574913,0.9550923357099248,0.20710152240504406,0,1
7,cmp_bd,574851,0.22475563232907309,0.4174216587235586,0,1
8,cmp_bm,574851,0.4886361857246487,0.49987128182816276,0,1
9,cmp_by,574851,0.22266639529199742,0.416036504164562,0,1


In [ ]:
summary_p.shape
summary_p = summary_p.rename_axis(None, axis=1)
summary_p


,fields,count,mean,stddev,min,max
0,id_1,574913,33271.962171667714,23622.66942593358,1,99894
1,id_2,574913,66564.6636865056,23642.00230967225,6,100000
2,cmp_fname_c1,574811,0.7127592938252765,0.3889286452463553,0.0,1.0
3,cmp_fname_c2,10325,0.8977586763518972,0.2742577520430534,0.0,1.0
4,cmp_lname_c1,574913,0.31557245780987964,0.3342494687554251,0.0,1.0
5,cmp_lname_c2,239,0.32691554145529067,0.37830920205406704,0.0,1.0
6,cmp_sex,574913,0.9550923357099248,0.20710152240504406,0,1
7,cmp_bd,574851,0.22475563232907309,0.4174216587235586,0,1
8,cmp_bm,574851,0.4886361857246487,0.49987128182816276,0,1
9,cmp_by,574851,0.22266639529199742,0.416036504164562,0,1


In [ ]:
summaryT = spark.createDataFrame(summary_p)

In [ ]:
summaryT.printSchema()

root
 |-- fields: string (nullable = true)
 |-- count: string (nullable = true)
 |-- mean: string (nullable = true)
 |-- stddev: string (nullable = true)
 |-- min: string (nullable = true)
 |-- max: string (nullable = true)



In [ ]:
from pyspark.sql.types import DoubleType
for c in summaryT.columns:
  if c == 'fields':
    continue
  summaryT = summaryT.withColumn(c,summaryT[c].cast(DoubleType()))

summaryT.printSchema()

root
 |-- fields: string (nullable = true)
 |-- count: double (nullable = true)
 |-- mean: double (nullable = true)
 |-- stddev: double (nullable = true)
 |-- min: double (nullable = true)
 |-- max: double (nullable = true)



In [ ]:
from pyspark.sql import DataFrame
def pivot_summary(desc):
  descP = desc.toPandas()
  descP = descP.set_index('summary').transpose().reset_index()
  descP = descP.rename(columns = {'index':'fields'})
  descT = spark.createDataFrame(descP)
  for c in descT.columns:
    if(c == 'fields'):
      continue
    else:
      descT = descT.withColumn(c, descT[c].cast(DoubleType()))

  return descT

In [ ]:
matchs_summary = pivot_summary(matchs_summary)
missed_summary = pivot_summary(missed_summary)

In [ ]:
matchs_summary.show()
missed_summary.show()

+------------+------+------------------+--------------------+-----------------+-------+
|      fields| count|              mean|              stddev|              min|    max|
+------------+------+------------------+--------------------+-----------------+-------+
|        id_1|2093.0| 34440.86956521739|  21743.893698458884|              5.0|98459.0|
|        id_2|2093.0| 50889.32345914955|    24382.2927756227|              6.0|99897.0|
|cmp_fname_c1|2091.0|0.9970329792424486| 0.03979189523588238|              0.0|    1.0|
|cmp_fname_c2| 128.0|0.9955357142857143| 0.05050762722761048|0.428571428571429|    1.0|
|cmp_lname_c1|2093.0| 0.997560095051734|0.039218983113320234|              0.0|    1.0|
|cmp_lname_c2|  53.0| 0.960167714884696| 0.17815710016004582|              0.0|    1.0|
|     cmp_sex|2093.0|0.9894887720974678| 0.10200839997032067|              0.0|    1.0|
|      cmp_bd|2093.0|0.9952221691352127| 0.06897301025106671|              0.0|    1.0|
|      cmp_bm|2093.0|0.997611084

# Decision Tree

In [ ]:
! pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=c8939dad1965761d7caf26a4137ebb5cf3d79cbe7ed815354b548614d9be92b3
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [ ]:
import pyspark
from  pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.ml.feature import VectorAssembler


In [ ]:
! unzip covtype

Archive:  covtype.zip
  inflating: covtype.data            


In [ ]:
spark  = SparkSession.builder.config("saprk.driver.memory", '4g').appName('decisionTree').getOrCreate()

In [ ]:
data_without_header = spark.read.option("inferSchema", True)\
                      .option("header", False).csv("covtype.data")
data_without_header.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- _c1: integer (nullable = true)
 |-- _c2: integer (nullable = true)
 |-- _c3: integer (nullable = true)
 |-- _c4: integer (nullable = true)
 |-- _c5: integer (nullable = true)
 |-- _c6: integer (nullable = true)
 |-- _c7: integer (nullable = true)
 |-- _c8: integer (nullable = true)
 |-- _c9: integer (nullable = true)
 |-- _c10: integer (nullable = true)
 |-- _c11: integer (nullable = true)
 |-- _c12: integer (nullable = true)
 |-- _c13: integer (nullable = true)
 |-- _c14: integer (nullable = true)
 |-- _c15: integer (nullable = true)
 |-- _c16: integer (nullable = true)
 |-- _c17: integer (nullable = true)
 |-- _c18: integer (nullable = true)
 |-- _c19: integer (nullable = true)
 |-- _c20: integer (nullable = true)
 |-- _c21: integer (nullable = true)
 |-- _c22: integer (nullable = true)
 |-- _c23: integer (nullable = true)
 |-- _c24: integer (nullable = true)
 |-- _c25: integer (nullable = true)
 |-- _c26: integer (nullable = true)
 |-- _

In [ ]:
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import col
colnames = ["Elevation", "Aspect", "Slope", \
            "Horizontal_Distance_To_Hydrology", \
"Vertical_Distance_To_Hydrology", "Horizontal_Distance_To_Roadways",
"Hillshade_9am", "Hillshade_Noon", "Hillshade_3pm", \
"Horizontal_Distance_To_Fire_Points"] + \
[f"Wilderness_Area_{i}" for i in range(4)] + \
[f"Soil_Type_{i}" for i in range(40)] + \
["Cover_Type"]
data = data_without_header.toDF(*colnames).withColumn("Cover_Type",col("Cover_Type").cast(DoubleType()))
data.head()

Row(Elevation=2596, Aspect=51, Slope=3, Horizontal_Distance_To_Hydrology=258, Vertical_Distance_To_Hydrology=0, Horizontal_Distance_To_Roadways=510, Hillshade_9am=221, Hillshade_Noon=232, Hillshade_3pm=148, Horizontal_Distance_To_Fire_Points=6279, Wilderness_Area_0=1, Wilderness_Area_1=0, Wilderness_Area_2=0, Wilderness_Area_3=0, Soil_Type_0=0, Soil_Type_1=0, Soil_Type_2=0, Soil_Type_3=0, Soil_Type_4=0, Soil_Type_5=0, Soil_Type_6=0, Soil_Type_7=0, Soil_Type_8=0, Soil_Type_9=0, Soil_Type_10=0, Soil_Type_11=0, Soil_Type_12=0, Soil_Type_13=0, Soil_Type_14=0, Soil_Type_15=0, Soil_Type_16=0, Soil_Type_17=0, Soil_Type_18=0, Soil_Type_19=0, Soil_Type_20=0, Soil_Type_21=0, Soil_Type_22=0, Soil_Type_23=0, Soil_Type_24=0, Soil_Type_25=0, Soil_Type_26=0, Soil_Type_27=0, Soil_Type_28=1, Soil_Type_29=0, Soil_Type_30=0, Soil_Type_31=0, Soil_Type_32=0, Soil_Type_33=0, Soil_Type_34=0, Soil_Type_35=0, Soil_Type_36=0, Soil_Type_37=0, Soil_Type_38=0, Soil_Type_39=0, Cover_Type=5.0)

In [ ]:
(train_data,test_data) = data.randomSplit([.9, .1])
train_data.cache()
test_data.cache()

DataFrame[Elevation: int, Aspect: int, Slope: int, Horizontal_Distance_To_Hydrology: int, Vertical_Distance_To_Hydrology: int, Horizontal_Distance_To_Roadways: int, Hillshade_9am: int, Hillshade_Noon: int, Hillshade_3pm: int, Horizontal_Distance_To_Fire_Points: int, Wilderness_Area_0: int, Wilderness_Area_1: int, Wilderness_Area_2: int, Wilderness_Area_3: int, Soil_Type_0: int, Soil_Type_1: int, Soil_Type_2: int, Soil_Type_3: int, Soil_Type_4: int, Soil_Type_5: int, Soil_Type_6: int, Soil_Type_7: int, Soil_Type_8: int, Soil_Type_9: int, Soil_Type_10: int, Soil_Type_11: int, Soil_Type_12: int, Soil_Type_13: int, Soil_Type_14: int, Soil_Type_15: int, Soil_Type_16: int, Soil_Type_17: int, Soil_Type_18: int, Soil_Type_19: int, Soil_Type_20: int, Soil_Type_21: int, Soil_Type_22: int, Soil_Type_23: int, Soil_Type_24: int, Soil_Type_25: int, Soil_Type_26: int, Soil_Type_27: int, Soil_Type_28: int, Soil_Type_29: int, Soil_Type_30: int, Soil_Type_31: int, Soil_Type_32: int, Soil_Type_33: int, S